## Microsoft Malware Dataloader

The malware industry continues to be a well-organized, well-funded market dedicated to evading traditional security measures. Once a computer is infected by malware, criminals can hurt consumers and enterprises in many ways.

https://www.kaggle.com/c/microsoft-malware-prediction


### Goal: FInd the Malware!

This is a tabular competition and my first jump back into a Kaggle competition for 8 days of fury.
The Microsoft Malware Loader needs to run before this one is. This notebook creates the dataset. 

In [1]:
import pickle
import gc
from fastai.tabular import *
torch.cuda.set_device(0)
pd.options.display.max_rows = 5
pd.options.display.max_columns = 5

In [2]:
## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
## Insert whatever audio file you want above

In [3]:
#If dataframe category has less than X instances; replace with a default value. 

def Combiner(df, col, val_lower, otherval):
    top_cat = list(df[col].value_counts(dropna=False).index[:])
    df_split=df.loc[df[col].isin(top_cat)]
    
    a = df_split[col].value_counts()
    m = df_split[col].isin(a.index[a<val_lower])
    m = list(m)
    df.loc[m, col] = otherval
        
    return df

In [4]:
# Reducing memory usage is amazingly helpful for this competetion
import pandas as pd
import numpy as np

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object and col_type.name != 'category':
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [5]:
## Rename some items to better understand their uniqueness
def rename_edition(x):
    x = x.lower()
    if 'core' in x:
        return 'Core'
    elif 'pro' in x:
        return 'pro'
    elif 'enterprise' in x:
        return 'Enterprise'
    elif 'server' in x:
        return 'Server'
    elif 'home' in x:
        return 'Home'
    elif 'education' in x:
        return 'Education'
    elif 'cloud' in x:
        return 'Cloud'
    else:
        return x

In [6]:
# To successfully import we need to reduce the size of the variables
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int16',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float64', # was 'float32'
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32', # was 'float16'
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float32', # was 'float16'
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float64', # was 'float32'
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float64', # was 'float32'
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32', # was 'float16'
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float64', # was 'float32'
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        #'HasDetections':                                        'int8'
        'HasDetections':                                        'float16'
        }

In [7]:
# Its the path
path = Path('/home/jd/data/microsoft')

In [8]:
# Split apart the date. I stripped lots of it due to it probably not helping

def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek']
    #, 'Dayofyear', 'Is_month_end'], 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    #df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [9]:
# Feature Engineering borrowed from Andrew Lukyanenko 
# https://www.kaggle.com/artgor/is-this-malware-eda-fe-and-lgb-updated
def fe(df):
    
    #Splitting everything out
    df['OsBuildLab'] = df['OsBuildLab'].cat.add_categories(['0.0.0.0.0-0'])
    df['OsBuildLab'] = df['OsBuildLab'].fillna('0.0.0.0.0-0')
    
    df['EngineVersion_2'] = df['EngineVersion'].apply(lambda x: x.split('.')[2]).astype('category')
    df['EngineVersion_3'] = df['EngineVersion'].apply(lambda x: x.split('.')[3]).astype('category')

    df['AppVersion_1'] = df['AppVersion'].apply(lambda x: x.split('.')[1]).astype('category')
    df['AppVersion_2'] = df['AppVersion'].apply(lambda x: x.split('.')[2]).astype('category')
    df['AppVersion_3'] = df['AppVersion'].apply(lambda x: x.split('.')[3]).astype('category')

    df['AvSigVersion_0'] = df['AvSigVersion'].apply(lambda x: x.split('.')[0]).astype('category')
    df['AvSigVersion_1'] = df['AvSigVersion'].apply(lambda x: x.split('.')[1]).astype('category')
    df['AvSigVersion_2'] = df['AvSigVersion'].apply(lambda x: x.split('.')[2]).astype('category')

    df['OsBuildLab_0'] = df['OsBuildLab'].apply(lambda x: x.split('.')[0]).astype('category')
    df['OsBuildLab_1'] = df['OsBuildLab'].apply(lambda x: x.split('.')[1]).astype('category')
    df['OsBuildLab_2'] = df['OsBuildLab'].apply(lambda x: x.split('.')[2]).astype('category')
    df['OsBuildLab_3'] = df['OsBuildLab'].apply(lambda x: x.split('.')[3]).astype('category')

    df['Census_OSVersion_0'] = df['Census_OSVersion'].apply(lambda x: x.split('.')[0]).astype('category')
    df['Census_OSVersion_1'] = df['Census_OSVersion'].apply(lambda x: x.split('.')[1]).astype('category')
    df['Census_OSVersion_2'] = df['Census_OSVersion'].apply(lambda x: x.split('.')[2]).astype('category')
    df['Census_OSVersion_3'] = df['Census_OSVersion'].apply(lambda x: x.split('.')[3]).astype('category')

    
    # https://www.kaggle.com/adityaecdrid/simple-feature-engineering-xd
    df['primary_drive_c_ratio'] = df['Census_SystemVolumeTotalCapacity']/ df['Census_PrimaryDiskTotalCapacity']
    #df['non_primary_drive_MB'] = df['Census_PrimaryDiskTotalCapacity'] - df['Census_SystemVolumeTotalCapacity']
    
    df['aspect_ratio'] = df['Census_InternalPrimaryDisplayResolutionHorizontal']/ df['Census_InternalPrimaryDisplayResolutionVertical']
    #df['monitor_dims'] = df['Census_InternalPrimaryDisplayResolutionHorizontal'].astype(str) + '*' + df['Census_InternalPrimaryDisplayResolutionVertical'].astype('str')
    #df['monitor_dims'] = df['monitor_dims'].astype('category')

    df['dpi'] = ((df['Census_InternalPrimaryDisplayResolutionHorizontal']**2 + df['Census_InternalPrimaryDisplayResolutionVertical']**2)**.5)/(df['Census_InternalPrimaryDiagonalDisplaySizeInInches'])

    #df['dpi_square'] = df['dpi'] ** 2

    df['MegaPixels'] = (df['Census_InternalPrimaryDisplayResolutionHorizontal'] * df['Census_InternalPrimaryDisplayResolutionVertical'])/1e6

    #df['Screen_Area'] = (df['aspect_ratio']* (df['Census_InternalPrimaryDiagonalDisplaySizeInInches']**2))/(df['aspect_ratio']**2 + 1)

    df['ram_per_processor'] = df['Census_TotalPhysicalRAM']/ df['Census_ProcessorCoreCount']

    df['new_num_0'] = df['Census_InternalPrimaryDiagonalDisplaySizeInInches'] / df['Census_ProcessorCoreCount']
    
    df['new_num_1'] = df['Census_ProcessorCoreCount'] * df['Census_InternalPrimaryDiagonalDisplaySizeInInches']
    
    df['Census_IsFlightingInternal'] = df['Census_IsFlightingInternal'].fillna(1)
    df['Census_ThresholdOptIn'] = df['Census_ThresholdOptIn'].fillna(1)
    df['Census_IsWIMBootEnabled'] = df['Census_IsWIMBootEnabled'].fillna(1)
    df['Wdft_IsGamer'] = df['Wdft_IsGamer'].fillna(0) 
    
    # 
    df.loc[df['AVProductsInstalled'].isin([1, 2]) == False, 'AVProductsInstalled'] = 3
    df.loc[df['OrganizationIdentifier'].isin([27, 18]) == False, 'OrganizationIdentifier'] = 48
    df['OrganizationIdentifier'] = df['OrganizationIdentifier'].astype('category')
    
    df['Census_OSSkuName'] = df['Census_OSSkuName'].astype(str)
    df['Census_OSSkuName'] = df['Census_OSSkuName'].apply(rename_edition)
    df['Census_OSSkuName'] = df['Census_OSSkuName'].astype('category')


    df.loc[df['Census_ProcessorCoreCount'].isin([2, 4, 8, 12]) == False, 'Census_ProcessorCoreCount'] = 1

    top_10 = df['Census_TotalPhysicalRAM'].value_counts(dropna=False, normalize=True).cumsum().index[:10]
    df.loc[df['Census_TotalPhysicalRAM'].isin(top_10) == False, 'Census_TotalPhysicalRAM'] = 1000

    def group_battery(x):
        x = x.lower()
        if 'li' in x:
            return 1
        else:
            return 0
    
    df['Census_InternalBatteryType'] = df['Census_InternalBatteryType'].apply(group_battery)
    
    
    df.loc[df['SmartScreen'].isnull(), 'SmartScreen'] = 'ExistsNotSet'
    df.loc[df['SmartScreen'].isin(['RequireAdmin', 'ExistsNotSet', 'Off', 'Warn']) == False, 'SmartScreen'] = 'Prompt'
    df['SmartScreen'] = df['SmartScreen'].cat.remove_unused_categories()

    df.loc[df['Census_PrimaryDiskTypeName'].isin(['HDD', 'SSD']) == False, 'Census_PrimaryDiskTypeName'] = 'UNKNOWN'
    df['Census_PrimaryDiskTypeName'] = df['Census_PrimaryDiskTypeName'].cat.remove_unused_categories()

    df.loc[df['Census_ProcessorManufacturerIdentifier'].isin([5.0, 1.0]) == False, 'Census_ProcessorManufacturerIdentifier'] = 0.0
    df['Census_ProcessorManufacturerIdentifier'] = df['Census_ProcessorManufacturerIdentifier'].astype('category')
    
    df.loc[df['Census_PowerPlatformRoleName'].isin(['Mobile', 'Desktop', 'Slate']) == False, 'Census_PowerPlatformRoleName'] = 'UNKNOWN'
    df['Census_PowerPlatformRoleName'] = df['Census_PowerPlatformRoleName'].cat.remove_unused_categories()
    
    
    top_cats = list(df['Census_OSWUAutoUpdateOptionsName'].value_counts().index[:3])
    df.loc[df['Census_OSWUAutoUpdateOptionsName'].isin(top_cats) == False, 'Census_OSWUAutoUpdateOptionsName'] = 'Off'
    df['Census_OSWUAutoUpdateOptionsName'] = df['Census_OSWUAutoUpdateOptionsName'].cat.remove_unused_categories()
    
    df.loc[df['Census_GenuineStateName'] == 'UNKNOWN', 'Census_GenuineStateName'] = 'OFFLINE'
    df['Census_GenuineStateName'] = df['Census_GenuineStateName'].cat.remove_unused_categories()
    
    
    df.loc[df['Census_ActivationChannel'].isin(['Retail', 'OEM:DM']) == False, 'Census_ActivationChannel'] = 'Volume:GVLK'
    df['Census_ActivationChannel'] = df['Census_ActivationChannel'].cat.remove_unused_categories()
    
    
    # Drop for memory
    

    df.drop('Census_InternalPrimaryDisplayResolutionHorizontal', axis=1)
    df.drop('Census_OSUILocaleIdentifier', axis=1)
    df.drop('AppVersion_3',axis=1)
    
    remove_cols = ['PuaMode', 'Census_ProcessorClass', 
                   'Census_IsWIMBootEnabled', 'IsBeta', 
                   'Census_IsFlightsDisabled', 'Census_IsFlightingInternal', 
                   'AutoSampleOptIn', 'Census_ThresholdOptIn', 'SMode', 
                   'Census_IsPortableOperatingSystem', 'Census_DeviceFamily', 
                   'UacLuaenable', 'Census_IsVirtualDevice', 'Platform', 
                   'Census_OSSkuName', 'Census_OSInstallLanguageIdentifier', 'Processor']
    df.drop(remove_cols, axis=1, inplace=True)
    
    gc.collect()
    return df

In [10]:
# Feature Engineering round #2
# I thought the number of alerts presented might help provde something... it did not.
def fe2(df):
    AVThreats = pd.read_csv(path/'AvSigversion_Threats.csv')
    AVTHigh=AVThreats[AVThreats['AlertLevel'] == 'high']
    AVTSevere=AVThreats[AVThreats['AlertLevel'] == 'severe']
    
    #Was just going to looke at High and Severe. Those are what we care about the most
    cv = pd.DataFrame(AVTHigh.groupby('AvSigVersion')['index'].count()).rename({'index':'AVTHigh'},axis=1)
    df = pd.merge(df,cv,on='AvSigVersion',how='left')
    df['AVTHigh'].fillna(0,inplace=True).astype('float16')

    cv = pd.DataFrame(AVTSevere.groupby('AvSigVersion')['index'].count()).rename({'index':'AVTSevere'},axis=1)
    df = pd.merge(df,cv,on='AvSigVersion',how='left')
    df['AVTSevere'].fillna(0,inplace=True).astype('float16')
    
    # The total number
    df['TotalAVT']=  df['AVTSevere'] + df['AVTHigh']
    df['TotalAVT'].fillna(0,inplace=True).astype('float16')

    return df

In [11]:
# Pulling data used for timestamps
# https://www.kaggle.com/cdeotte/external-data-malware-0-50
datedictAS = np.load(path/'AvSigVersionTimestamps.npy')[()]
datedictOS = np.load(path/'OSVersionTimestamps.npy')[()]

## Train the dataframe

In [12]:
# Pickles are faster at loading, so to save time I we turned them into pickles!

#df = pd.read_csv(path/'train.csv', dtype=dtypes)
#pickle.dump(df, open( "/home/jd/data/microsoft/microsoft-startdf-save.p", "wb" ) )
df=pickle.load(open( "/home/jd/data/microsoft/microsoft-startdf-save.p", "rb" ) )

In [13]:
# This did not help
#fe2(df)

In [14]:
# Feature Engineering for the train set
fe(df)

,MachineIdentifier,ProductName,...,new_num_0,new_num_1
0,0000028988387b115f69f31a3bf04f09,win8defender,...,4.725,75.599998
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,...,3.475,55.599998
...,...,...,...,...,...
8921481,fffffbbaaf5969ae4b93e7f3f6d7132f,win8defender,...,6.350,25.400000
8921482,ffffff75ba4f33d938ccfdb148b8ea16,win8defender,...,3.875,62.000000


In [15]:
# We want to remove columns that might not be helpful. 
removeCols=[]
for col in df.columns:
    rate = df[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.95:
        removeCols.append(col)

In [16]:
#I wanted to reduce the number of unique variables. IE if a city only showed up once, it was likely a bad indicator. 
# We get to save a little bit of memory by doing this.

df['AvSigVersion'].nunique(),df['AVProductStatesIdentifier'].nunique(), df['CityIdentifier'].nunique(),df['Census_OEMNameIdentifier'].nunique(),df['Census_InternalBatteryNumberOfCharges'].nunique(),df['Census_FirmwareVersionIdentifier'].nunique()

(8531, 28970, 107366, 3832, 41088, 50494)

In [17]:
#df2=df[['AvSigVersion', 
#'AVProductStatesIdentifier',
#'CityIdentifier',
#'Census_OEMNameIdentifier',
#'Census_InternalBatteryNumberOfCharges',
#'Census_FirmwareVersionIdentifier',
#'HasDetections']]
cutoff=100

Combiner(df,'AvSigVersion',cutoff,'0.0.0.0')
Combiner(df,'AVProductStatesIdentifier',cutoff, '0.0')
Combiner(df,'CityIdentifier',cutoff,'0.0')
Combiner(df,'Census_OEMNameIdentifier',cutoff,'3')
Combiner(df,'Census_InternalBatteryNumberOfCharges',cutoff,'50')
Combiner(df,'Census_FirmwareVersionIdentifier',cutoff,'50')
df['AvSigVersion'].nunique(),df['AVProductStatesIdentifier'].nunique(), df['CityIdentifier'].nunique(),df['Census_OEMNameIdentifier'].nunique(),df['Census_InternalBatteryNumberOfCharges'].nunique(),df['Census_FirmwareVersionIdentifier'].nunique()

(2725, 28723, 103928, 3831, 40832, 49711)

In [18]:
#df.info()

In [19]:
# Date feature engineering. Rumors abound that the testset were later. Tried to have something taking the latest date
#from the set as the most recent release and the number of days since the last update. Afterall this is a key
# item in security. 

def dateFE(df, col,datedict):
    #Date information
    label='Date'+str(col)
    df['Date'+str(col)] = df[col].map(datedict)
    add_datepart(df, 'Date'+str(col), drop=False)
    datetime =  df['Date'+str(col)].max()
    print(datetime)
    df['DaysElapsed'+col]=  ((datetime)- df['Date'+str(col)]).dt.days
    df=df.drop(label,axis =1)
    print('Variable is '+str(label))
    return df

In [20]:
#Should be 2018-09-25 23:56:00
dateFE(df,'AvSigVersion',datedictAS)
df=df.drop('DateAvSigVersion',axis =1)

2018-09-25 23:56:00
Variable is DateAvSigVersion


In [21]:
#get newest date 2018-10-02 00:00:00
dateFE(df,'Census_OSVersion',datedictOS)
df=df.drop('DateCensus_OSVersion',axis =1)

2018-10-02 00:00:00
Variable is DateCensus_OSVersion


In [22]:
#reducing that memory!
reduce_mem_usage(df)

Memory usage of dataframe is 3187.85 MB
Memory usage after optimization is: 1930.77 MB
Decreased by 39.4%


,MachineIdentifier,ProductName,...,DateCensus_OSVersionDayofweek,DaysElapsedCensus_OSVersion
0,0000028988387b115f69f31a3bf04f09,win8defender,...,1.0,84.0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,...,4.0,165.0
...,...,...,...,...,...
8921481,fffffbbaaf5969ae4b93e7f3f6d7132f,win8defender,...,1.0,448.0
8921482,ffffff75ba4f33d938ccfdb148b8ea16,win8defender,...,1.0,49.0


In [23]:
## Save the pickle
pickle.dump(df, open( "/home/jd/data/microsoft/microsoft-df-save.p", "wb" ) )

In [24]:
# gc just in case
gc.collect()

7

## Test Data
Do the same thing for test data!

In [25]:
#test_df = pd.read_csv(path/'test.csv', dtype=dtypes)
#pickle.dump(test_df, open( "/home/jd/data/microsoft/microsoft-starttestdf-save.p", "wb" ) )
test_df=pickle.load(open( "/home/jd/data/microsoft/microsoft-starttestdf-save.p", "rb" ) )

In [26]:
fe(test_df)

,MachineIdentifier,ProductName,...,new_num_0,new_num_1
0,0000010489e3af074adeac69c53e555e,win8defender,...,3.875,62.000000
1,00000176ac758d54827acd545b6315a5,win8defender,...,3.875,62.000000
...,...,...,...,...,...
7853251,fffffad7b6c8196ec5cae634406c0d4f,win8defender,...,3.875,62.000000
7853252,fffffbd305a90eb0f93ee4f30a39c736,win8defender,...,7.800,31.200001


In [27]:
#fe2(test_df)

In [28]:
test_df.drop(removeCols, axis=1)

,MachineIdentifier,EngineVersion,...,new_num_0,new_num_1
0,0000010489e3af074adeac69c53e555e,1.1.15400.5,...,3.875,62.000000
1,00000176ac758d54827acd545b6315a5,1.1.15400.4,...,3.875,62.000000
...,...,...,...,...,...
7853251,fffffad7b6c8196ec5cae634406c0d4f,1.1.15400.4,...,3.875,62.000000
7853252,fffffbd305a90eb0f93ee4f30a39c736,1.1.15400.5,...,7.800,31.200001


In [29]:
Combiner(test_df,'AvSigVersion',cutoff,'0.0.0.0')
Combiner(test_df,'AVProductStatesIdentifier',cutoff, '0.0')
Combiner(test_df,'CityIdentifier',cutoff,'0.0')
Combiner(test_df,'Census_OEMNameIdentifier',cutoff,'3')
Combiner(test_df,'Census_InternalBatteryNumberOfCharges',cutoff,'50')
Combiner(test_df,'Census_FirmwareVersionIdentifier',cutoff,'50')

,MachineIdentifier,ProductName,...,new_num_0,new_num_1
0,0000010489e3af074adeac69c53e555e,win8defender,...,3.875,62.000000
1,00000176ac758d54827acd545b6315a5,win8defender,...,3.875,62.000000
...,...,...,...,...,...
7853251,fffffad7b6c8196ec5cae634406c0d4f,win8defender,...,3.875,62.000000
7853252,fffffbd305a90eb0f93ee4f30a39c736,win8defender,...,7.800,31.200001


In [30]:
# Should be
dateFE(test_df,'AvSigVersion',datedictAS)
test_df=test_df.drop('DateAvSigVersion',axis =1)

2018-11-25 23:28:00
Variable is DateAvSigVersion


In [31]:
# Should be
dateFE(test_df,'Census_OSVersion',datedictOS)
test_df=test_df.drop('DateCensus_OSVersion',axis =1)

2018-11-16 00:00:00
Variable is DateCensus_OSVersion


In [32]:
reduce_mem_usage(test_df)

Memory usage of dataframe is 2814.58 MB
Memory usage after optimization is: 1723.55 MB
Decreased by 38.8%


,MachineIdentifier,ProductName,...,DateCensus_OSVersionDayofweek,DaysElapsedCensus_OSVersion
0,0000010489e3af074adeac69c53e555e,win8defender,...,1.0,38.0
1,00000176ac758d54827acd545b6315a5,win8defender,...,1.0,94.0
...,...,...,...,...,...
7853251,fffffad7b6c8196ec5cae634406c0d4f,win8defender,...,1.0,521.0
7853252,fffffbd305a90eb0f93ee4f30a39c736,win8defender,...,2.0,849.0


In [33]:
pickle.dump(test_df, open( "/home/jd/data/microsoft/microsoft-test.p", "wb" ) )

## Relax Data
Lots of help here to take away extremes in the data. 
https://www.kaggle.com/artgor/is-this-malware-eda-fe-and-lgb-updated

In [34]:
# lets take the most recent
df=pickle.load(open( "/home/jd/data/microsoft/microsoft-df-save.p", "rb" ) )
test_df = pickle.load(open( "/home/jd/data/microsoft/microsoft-test.p", "rb" ) )

In [35]:
# Bad value here
df['AvSigVersion_1']=df['AvSigVersion_1'].replace('2&#x17;3', 0)

In [36]:
# I wanted to make a validation set which mainly had items later on. This makes sense if viewing it as a time series.
# it is not a time series....
df['AvSigVersion_1']=df['AvSigVersion_1'].astype('float16')
#https://www.kaggle.com/cdeotte/time-split-validation-malware-0-68
#df_trainC = df[ df['AvSigVersion_1']<275 ]
df_trainD = df[ df['AvSigVersion_1']>=275 ]
valid_idx=random.sample(df_trainD.index.tolist(), int(len(df_trainD)*.5))
pickle.dump(valid_idx, open( "/home/jd/data/microsoft/valid_idx.p", "wb" ) )
df['AvSigVersion_1']=df['AvSigVersion_1'].astype('category')

In [38]:
def factor_data(df_train, df_test, col):
    df_comb = pd.concat([df_train[col],df_test[col]],axis=0)
    df_comb,_ = df_comb.factorize(sort=True)
    # MAKE SMALLEST LABEL 1, RESERVE 0
    df_comb += 1
    # MAKE NAN LARGEST LABEL
    df_comb = np.where(df_comb==0, df_comb.max()+1, df_comb)
    df_train[col] = df_comb[:len(df_train)]
    df_test[col] = df_comb[len(df_train):]
    del df_comb

In [37]:
# Walks through all the columns to relax data. We dont want to do this on 3 items.
cols = [x for x in df.columns if x not in ['MachineIdentifier','HasDetections','AvSigVersion_1']]

def relax_data(df_train, df_test, col):
    cv1 = pd.DataFrame(df_train[col].value_counts().reset_index().rename({col:'train'},axis=1))
    cv2 = pd.DataFrame(df_test[col].value_counts().reset_index().rename({col:'test'},axis=1))
    cv3 = pd.merge(cv1,cv2,on='index',how='outer')
    cv3['train'].fillna(0,inplace=True)
    cv3['test'].fillna(0,inplace=True)
    factor = len(df_test)/len(df_train)
    cv3['remove'] = False
    cv3['remove'] = cv3['remove'] | (cv3['train'] < len(df_train)/9000)
    cv3['remove'] = cv3['remove'] | (factor*cv3['train'] < cv3['test']/5)
    cv3['remove'] = cv3['remove'] | (factor*cv3['train'] > 5*cv3['test'])
    cv3['new'] = cv3.apply(lambda x: x['index'] if x['remove']==False else 0,axis=1)
    cv3['new'],_ = cv3['new'].factorize(sort=True)
    cv3.set_index('index',inplace=True)
    cc = cv3['new'].to_dict()
    df_train[col] = df_train[col].map(cc)
    reduce_memory(df_train,col)
    df_test[col] = df_test[col].map(cc)
    reduce_memory(df_test,col)
    

In [39]:
for col in cols: factor_data(df, test_df, col)

In [40]:
reduce_mem_usage(df)
reduce_mem_usage(test_df)

Memory usage of dataframe is 7118.05 MB
Memory usage after optimization is: 1706.84 MB
Decreased by 76.0%
Memory usage of dataframe is 5969.09 MB
Memory usage after optimization is: 1525.80 MB
Decreased by 74.4%


,MachineIdentifier,ProductName,...,DateCensus_OSVersionDayofweek,DaysElapsedCensus_OSVersion
0,0000010489e3af074adeac69c53e555e,2,...,2,17
1,00000176ac758d54827acd545b6315a5,2,...,2,41
...,...,...,...,...,...
7853251,fffffad7b6c8196ec5cae634406c0d4f,2,...,2,136
7853252,fffffbd305a90eb0f93ee4f30a39c736,2,...,3,193


In [41]:
# I feel this speeds everything up
valid_idx.sort()

In [42]:
# Save it all
pickle.dump(test_df, open( "/home/jd/data/microsoft/microsoft-test.p", "wb" ) )
pickle.dump(df, open( "/home/jd/data/microsoft/microsoft-df-save.p", "wb" ) )
pickle.dump(valid_idx, open( "/home/jd/data/microsoft/valid_idx", "wb" ) )

## Categories and Cont Variables

In [43]:
# Continous variables to use
cont_names = ['Census_SystemVolumeTotalCapacity'
             ,'primary_drive_c_ratio']

In [44]:
# Get the categorical variables
cat_names = list(set(test_df.columns) - set(cont_names))

In [45]:
cat_names.remove('MachineIdentifier')

In [46]:
#Just some checks to make sure I didn't miss something
len(cat_names),len(cont_names)

(97, 2)

In [47]:
len(list(test_df.columns))

100

In [48]:
set(cat_names) & set(cont_names)

set()

In [49]:
set(list(df.columns))-set(list(test_df.columns))

{'HasDetections'}

## Save Pickle

In [50]:
pickle.dump(cat_names, open( "/home/jd/data/microsoft/microsoft-cat-save.p", "wb" ) )
pickle.dump(cont_names, open( "/home/jd/data/microsoft/microsoft-cont-save.p", "wb" ) )

In [51]:
allDone()

In [ ]:
#pd.options.display.max_rows = 2000
#pd.options.display.max_columns = 1000
#df